In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import itertools
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [3]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')
df[:2]

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df.dropna(axis=0, inplace=True)
df.reset_index(inplace=True)

In [6]:
df.shape

(18285, 6)

In [7]:
df.isnull().sum()

index     0
id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
# x = df.iloc[:,1:4]
x = df.drop(['label','id'],axis=1)
x[:2]

,index,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...


In [9]:
y = df['label']
y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [10]:
x.shape, y.shape

((18285, 4), (18285,))

In [11]:
msgs = df.copy()
msgs.shape

(18285, 6)

In [12]:
ps = PorterStemmer()

In [13]:
#Dropping the null value examples
df = df.dropna()
df.isnull().values.any()

False

In [14]:
corpus=[]
for i in range(len(msgs)):
    r = msgs['title'][i]
    r = re.sub('[^a-zA-Z]',' ',r)
    r = r.lower()
    r = r.split()
    r = [ps.stem(w) for w in r if not w in stopwords.words('english')]
    r = ' '.join(r)
    corpus.append(r)
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

In [15]:
voc_size=10000

In [16]:
# One hot representation
ohr = [one_hot(word, voc_size) for word in corpus]
ohr[:5]

[[8072, 770, 2302, 6178, 9148, 698, 6616, 7263, 2863, 3107],
 [2281, 5403, 7818, 5428, 2108, 4196, 2348],
 [6877, 3078, 2191, 1359],
 [9250, 8357, 5149, 5087, 8393, 521],
 [8893, 2108, 1762, 108, 2485, 278, 2108, 2303, 2389, 5106]]

In [17]:
sent_length = 40
emb = pad_sequences(ohr, padding='pre', maxlen=sent_length)
emb[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0, 8072,  770, 2302,
       6178, 9148,  698, 6616, 7263, 2863, 3107], dtype=int32)

In [18]:
# Creating model
emb_features=40
model = Sequential()
model.add(Embedding(voc_size, emb_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 40, 40)            400000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [19]:
xf = np.array(emb)
yf = np.array(y)

In [20]:
xtr, xte, ytr, yte = tts(xf, yf, random_state=0, test_size=.2)

In [21]:
# Model training
model.fit(xtr, ytr, validation_data=(xte,yte), epochs=10, batch_size=128)

Epoch 1/10
115/115 [==============================] - 12s 81ms/step - loss: 0.4014 - accuracy: 0.8135 - val_loss: 0.2042 - val_accuracy: 0.9155
Epoch 2/10
115/115 [==============================] - 9s 79ms/step - loss: 0.1436 - accuracy: 0.9448 - val_loss: 0.1732 - val_accuracy: 0.9292
Epoch 3/10
115/115 [==============================] - 9s 75ms/step - loss: 0.0883 - accuracy: 0.9700 - val_loss: 0.1990 - val_accuracy: 0.9245
Epoch 4/10
115/115 [==============================] - 9s 75ms/step - loss: 0.0599 - accuracy: 0.9800 - val_loss: 0.2194 - val_accuracy: 0.9232
Epoch 5/10
115/115 [==============================] - 9s 79ms/step - loss: 0.0360 - accuracy: 0.9892 - val_loss: 0.2571 - val_accuracy: 0.9218
Epoch 6/10
115/115 [==============================] - 9s 77ms/step - loss: 0.0237 - accuracy: 0.9935 - val_loss: 0.2688 - val_accuracy: 0.9210
Epoch 7/10
115/115 [==============================] - 9s 76ms/step - loss: 0.0174 - accuracy: 0.9957 - val_loss: 0.2805 - val_accuracy: 0.922

In [22]:
p1 = model.predict(xte)
y1 = np.argmax(p1,axis=1)
accuracy_score(yte, y1)

115/115 [==============================] - 2s 10ms/step


0.5578342904019689

In [23]:
# Adding dropout
emb_features=40
model2 = Sequential()
model2.add(Embedding(voc_size, emb_features, input_length=sent_length))
model2.add(Dropout(.3))
model2.add(LSTM(100))
model2.add(Dropout(.3))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 40, 40)            400000    
                                                                 
 dropout (Dropout)           (None, 40, 40)            0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Model training
model2.fit(xtr, ytr, validation_data=(xte,yte), epochs=20, batch_size=128)

Epoch 1/20
115/115 [==============================] - 12s 81ms/step - loss: 0.4086 - accuracy: 0.8056 - val_loss: 0.2025 - val_accuracy: 0.9152
Epoch 2/20
115/115 [==============================] - 9s 77ms/step - loss: 0.1520 - accuracy: 0.9412 - val_loss: 0.1758 - val_accuracy: 0.9248
Epoch 3/20
115/115 [==============================] - 9s 79ms/step - loss: 0.1037 - accuracy: 0.9632 - val_loss: 0.1890 - val_accuracy: 0.9278
Epoch 4/20
115/115 [==============================] - 9s 76ms/step - loss: 0.0774 - accuracy: 0.9734 - val_loss: 0.2154 - val_accuracy: 0.9275
Epoch 5/20
115/115 [==============================] - 10s 89ms/step - loss: 0.0645 - accuracy: 0.9777 - val_loss: 0.2287 - val_accuracy: 0.9218
Epoch 6/20
115/115 [==============================] - 10s 84ms/step - loss: 0.0492 - accuracy: 0.9836 - val_loss: 0.2693 - val_accuracy: 0.9193
Epoch 7/20
115/115 [==============================] - 9s 78ms/step - loss: 0.0447 - accuracy: 0.9856 - val_loss: 0.2938 - val_accuracy: 0.9

In [ ]:
p2 = model2.predict(xte)
y2 = np.argmax(p2,axis=1)
accuracy_score(yte, y2)